In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import CabalTools
main = CabalTools.CabalTools(modules=['StellarManager'])

### Increase chances for higher colors (blue/purple)

In [ ]:
grade_chances = {
    (101, 1) : {'chance' : 60}, #Chance for worst grade while cheapest imprint
    (101, 2) : {'chance' : 25},
    (101, 3) : {'chance' : 10},
    (101, 4) : {'chance' : 3},
    (101, 5) : {'chance' : 2}, #Chance for the best grade while cheapest imprint
    (102, 1) : {'chance' : 40}, 
    (102, 2) : {'chance' : 30},
    (102, 3) : {'chance' : 15},
    (102, 4) : {'chance' : 10},
    (102, 5) : {'chance' : 5}, 
    (103, 1) : {'chance' : 40}, 
    (103, 2) : {'chance' : 25},
    (103, 3) : {'chance' : 15},
    (103, 4) : {'chance' : 13},
    (103, 5) : {'chance' : 7}, 
    (104, 1) : {'chance' : 25}, 
    (104, 2) : {'chance' : 25},
    (104, 3) : {'chance' : 20},
    (104, 4) : {'chance' : 20},
    (104, 5) : {'chance' : 10}, 
    #No cheapest grade with the best imprint
    (105, 2) : {'chance' : 30},
    (105, 3) : {'chance' : 35},
    (105, 4) : {'chance' : 20},
    (105, 5) : {'chance' : 15}, 
}
for k, v in grade_chances.items():
    main.StellarManager.modify_grade_chances(*k, **v, do_reinit=False, save_files=False)
main.StellarManager.normalize_grade_chances()

In [ ]:
main.StellarManager.reinit()
sink = main.StellarManager.preview(section_name='Stellar_Grade')

### Deleting the useless imprint options to improve general stats

In [ ]:
V_to_drop = [
    904, 905, 909, 910, 911,    #first line
    916, 917, 921, 922, 923,
    928, 929, 933, 934, 935,
    940, 941, 945, 946, 947,
    952, 953, 957, 958, 959,
    964, 965, 969,              #second line
    976, 977, 981,
    988, 989, 993,
    1000, 1001, 1005,
    1012, 1013, 1017,
    1024, 1025, 1029,           #3rd line
    1036, 1037, 1041,
    1048, 1049, 1053,
    1060, 1061, 1065,
    1072, 1073, 1077,
    1084, 1085, 1089,           #4th line
    1096, 1097, 1101,           
    1108, 1109, 1113,
    1120, 1121, 1125,
    1132, 1133, 1137,
    1144, 1145, 1149,           #5th line
    1156, 1157, 1161,
    1168, 1169, 1173,           
    1180, 1181, 1185,
    1192, 1193, 1197
]
for v in V_to_drop:
    main.StellarManager.truncate_force_value(v, do_reinit=False, save_files=False)

In [ ]:
main.StellarManager.reinit()
sink = main.StellarManager.preview(section_name=['Stellar_Forcecode', 'Stellar_Value'])

### Make only the best stat grade available to reduce RNG dependency

In [ ]:
for grade in (1, 2, 3, 4):
    for V_id in [int(x[0]) for x in list(sink[['V_Id']].values)]:
        main.StellarManager.remove_value_grade(V_id, grade, do_reinit=False, save_files=False)

In [ ]:
main.StellarManager.reinit()
sink = main.StellarManager.preview(section_name=['Stellar_Value'])

### Set uniform chance for each ForceCode

In [ ]:
Value_list = main.StellarManager.preview(section_name='Stellar_Forcecode', columns='Value', suppress=True)
Value_list = [int(x) for x in Value_list['Value'].values]
for val in Value_list:
    main.StellarManager.set_stat_percentage(val, 1)
main.StellarManager.normalize_stat_chances()

In [ ]:
main.StellarManager.reinit()
sink = main.StellarManager.preview(section_name=['Stellar_Forcecode'])

### Improve the imprint stats bonuses' values

In [ ]:
import pandas as pd

stat_grade_chances = main.StellarManager.preview(section_name=['Stellar_Value'], filter_key='V_Per', filter_val=0, filter_operator='>', suppress=True)
stat_chances = main.StellarManager.preview(section_name=['Stellar_Forcecode'], suppress=True)

current_config = pd.merge(
    stat_grade_chances, 
    stat_chances, 
    how='left', 
    left_on='V_Id', 
    right_on='Value')[['V_Id', 'Force_Code', 'Force_Code_Name', 'Value_x']].to_dict(orient='records')

stat_value_dict = {
    'Normal Attack DMG UP'      : 10,
    'All Attack UP'             : 60,
    'Cancel Ignore Penetration' : 30,
    'STR'                       : 30,
    'PVE Ignore Penetration'    : 50,
    'Defense'                   : 50,
    'Penetration'               : 20,
    'INT'                       : 30,
    'HP'                        : 30,
    'PVE Penetration'           : 25,
    'DEX'                       : 30,
    'Critical DMG.'             : 10,
    'PVE Defense'               : 50,
    'PVE Critical DMG'          : 10,
    'PVE DMG Reduction'         : 50
}

for record in current_config:
    new_config = {'V_Id' : record['V_Id'], 'V_Order' : 5, 'new_value' : stat_value_dict.get(record['Force_Code_Name'])}
    main.StellarManager.set_stat_value(**new_config, do_reinit=False, save_files=False)


In [ ]:
main.StellarManager.reinit()
sink = main.StellarManager.preview(section_name=['Stellar_Value'], filter_key='V_Per', filter_val=0, filter_operator='>')

In [ ]:
main.backup('stellar')
main.StellarManager.save()